# Week 1 Assignment

Install `pip3 install statsmodels`

In [118]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import edhec_risk_kit_206 as erk
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load the 49 Value Weighted industry portfolio returns. Limit the analysis to returns of the 49 VW portfolios from 1991 onwards (1991 included, 1990 not included, 2018 included). Also load the Fama French Research Factors over the same period. Use the Mkt-RF as in the lab notebooks to compute the CAPM betas.

In [119]:
ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)
ind49_rets.head()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
1926-07,0.0237,0.0012,NaN,-0.0519,0.0129,0.0865,0.0250,0.5021,-0.0048,0.0808,...,0.0770,0.0194,-0.2379,0.0007,0.0187,0.0461,-0.0054,0.0289,-0.0485,0.0520
1926-08,0.0223,0.0268,NaN,0.2703,0.0650,0.1681,-0.0076,0.4298,-0.0358,-0.0251,...,-0.0238,0.0488,0.0539,-0.0075,-0.0013,0.1183,0.0257,0.0530,-0.0057,0.0676
1926-09,-0.0057,0.0158,NaN,0.0402,0.0126,0.0833,0.0642,-0.0491,0.0073,-0.0051,...,-0.0554,0.0006,-0.0787,0.0025,-0.0056,-0.0175,0.0072,-0.0306,-0.0314,-0.0386
1926-10,-0.0046,-0.0368,NaN,-0.0331,0.0106,-0.0140,-0.0509,0.0537,-0.0468,0.0012,...,-0.0508,-0.0264,-0.1538,-0.0220,-0.0411,-0.1182,-0.0428,-0.0574,0.0207,-0.0849
1926-11,0.0675,0.0626,NaN,0.0729,0.0455,0.0000,0.0182,-0.0640,-0.0054,0.0187,...,0.0384,0.0160,0.0467,0.0652,0.0433,-0.0297,0.0358,0.0221,0.0492,0.0400


In [120]:
def capm(begin="1991", end="2018", industry="Beer"):
    ind_rets=ind49_rets[begin:end]
    fff = erk.get_fff_returns()[begin:end]
    ind_excess = ind_rets.loc[:,[industry]] - fff.loc[:,["RF"]].values
    mkt_excess = fff.loc[:, ["Mkt-RF"]]
    exp_var = mkt_excess.copy()
    exp_var["Constant"] = 1
    # return exp_var.head()
    lm = sm.OLS(ind_excess, exp_var).fit()
    return lm

## 1. Question 1
What is the CAPM (Single Factor) Beta when evaluated over the entire period (1991-2018) of Beer?

In [121]:
# capm().summary()
capm().params["Mkt-RF"] # The answer is 0.5295

0.5295417263363027

## 2. Question 2
Using the same data as the previous question, what is the CAPM Beta when evaluated over the entire period (1991-2018) of Steel?
Enter your answer as a number to at least 2 decimal places (e.g. 1.12)

In [122]:
capm(industry="Steel").params["Mkt-RF"] # The answer is 1.5546

1.5546095998085288

## 3. Question 3
Using the same data as the previous question, what is the CAPM Beta when evaluated over the 2013-2018 (both included) period of Beer?
Enter the answer as number up to at least 2 decimal places (e.g. 1.12)

In [123]:
capm(begin="2013").params["Mkt-RF"] # The answer is 0.5860

0.585960389056597

## 4. Question 4
Using the same data set as the previous question, what is the CAPM Beta when evaluated over the 2013-2018 (both included) period of Steel? 
Enter the answer as a number up to at least 2 decimal places (e.g. 1.12)

In [124]:
capm(begin="2013", industry="Steel").params["Mkt-RF"] # The answer is 1.4169

1.4169446523353808

## 5. Question 5
Using the same data as the previous question, which of the 49 industries had the highest CAPM Beta when evaluated over the 1991-1993 (both included) period? (Use the same industry names as in the files). Enter the name as a text string, and remember to exactly match the column headers in the data file.

In [125]:
max = 0
max_industry = ""
for industry in ind49_rets.columns:
    beta = capm(begin="1991", end="1993", industry=industry).params["Mkt-RF"]
    if (beta > max):
        max = beta
        max_industry = industry
print(max_industry, max)

Hlth 1.6151954971134441


## 6. Question 6
Using the same data as the previous question, which of the 49 industries had the lowest CAPM Beta when evaluated over the 1991-1993 (both included) period? (Use the same industry names as in the files) . Enter the answer as text and remember to exactly match the column headers in the data file.

In [126]:
min = sys.maxsize
min_industry = ""
for industry in ind49_rets.columns:
    beta = capm(begin="1991", end="1993", industry=industry).params["Mkt-RF"]
    if (beta < min):
        min = beta
        min_industry = industry
print(min_industry, min)

Gold -0.5532887150791107


## 7. Question 7
For the next 4 questions use the Full 3 Factor Fama-French model using the research data supplied in the following data file `F-F_Research_Data_Factors.csv` and the same 1991-2018 period you have just used for the previous questions.

In [127]:
ff_d = pd.read_csv("data/F-F_Research_Data_Factors.CSV", skiprows=3, skipfooter=(1211-1115), parse_dates=True, index_col=0)
ff_d.index = pd.to_datetime(ff_d.index, format="%Y%m").to_period('M')
ff_d.tail()

C:\Users\jlche\AppData\Local\Temp\ipykernel_12040\2007598248.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  ff_d = pd.read_csv("data/F-F_Research_Data_Factors.CSV", skiprows=3, skipfooter=(1211-1115), parse_dates=True, index_col=0)


,Mkt-RF,SMB,HML,RF
2018-08,3.44,1.23,-4.12,0.16
2018-09,0.06,-2.37,-1.34,0.15
2018-10,-7.68,-4.68,3.41,0.19
2018-11,1.69,-0.74,0.20,0.18
2018-12,-9.55,-2.61,-1.51,0.19


In [128]:
def fama_french(begin="1991", end="2018", industry="Beer"):
    ind_rets=ind49_rets[begin:end]
    fff = ff_d[begin:end]
    ind_excess = ind_rets.loc[:,[industry]] - fff.loc[:,["RF"]].values
    mkt_excess = fff.loc[:, ["Mkt-RF"]]
    exp_var = mkt_excess.copy()
    exp_var["Constant"] = 1
    exp_var["Value"] = fff.loc[:,['HML']]
    exp_var["Size"] = fff.loc[:,['SMB']]
    # return exp_var.head()
    lm = sm.OLS(ind_excess, exp_var).fit()
    return lm

Of the 49 industries, which displayed the highest Small Cap tilt when analyzed over the entire 1991-2018 period?
Enter your answer as a text string, and remember to use the exact same name as the column header in the text file

In [129]:
max = 0
max_industry = ""
for industry in ind49_rets.columns:
    beta = fama_french(begin="1991", end="2018", industry=industry).params["Size"]
    if (beta > max):
        max = beta
        max_industry = industry
print(max_industry, max)

FabPr 0.008694964281733879


## 8. Question 8
Using the same dataset and period as the previous question, of the 49 industries, which displayed the highest Large Cap tilt when analyzed over the entire period? 
Enter the name of the industry as text, using the exact same name as the column header in the data file.

In [130]:
min = sys.maxsize
min_industry = ""
for industry in ind49_rets.columns:
    beta = fama_french(begin="1991", end="2018", industry=industry).params["Size"]
    if (beta < min):
        min = beta
        min_industry = industry
print(min_industry, min)

Beer -0.0017766668540764958


## 9. Question 9
Using the same data as period as the previous question, of the 49 industries, which displayed the highest Value tilt when analyzed over the entire period?    
Enter the name of the industry as text, using the exact same name as the column header in the data file.

In [131]:
max = 0
max_industry = ""
for industry in ind49_rets.columns:
    beta = fama_french(begin="1991", end="2018", industry=industry).params["Value"]
    if (beta > max):
        max = beta
        max_industry = industry
print(max_industry, max)

Txtls 0.006082091479382075


## 10. Question 10
Using the same data set and period as the previous question, of the 49 industries, which displayed the highest Growth tilt when analyzed over the entire period?
Enter the name of the industry as text, using the exact same name as the column header in the data file.

In [132]:
min = sys.maxsize
min_industry = ""
for industry in ind49_rets.columns:
    beta = fama_french(begin="1991", end="2018", industry=industry).params["Value"]
    if (beta < min):
        min = beta
        min_industry = industry
print(min_industry, min)

Softw -0.012277727500611497
